# Reading silver table

In [0]:
from delta.tables import DeltaTable

In [0]:
df_silver = spark.sql("SELECT * FROM PARQUET.`abfss://silver@datalakeproject1goks.dfs.core.windows.net/car_sales`")

In [0]:
df_silver.display()

Reading all DIMS

In [0]:
df_dealer = spark.sql("SELECT* FROM cars_catalog.gold.dim_dealer")
df_branch = spark.sql("SELECT* FROM cars_catalog.gold.dim_branch")
df_model = spark.sql("SELECT* FROM cars_catalog.gold.dim_model")
df_date = spark.sql("SELECT* FROM cars_catalog.gold.dim_date")

Bringing keys to fact table

In [0]:
df_fact = df_silver.join(df_branch,df_silver['Branch_ID']==df_branch['Branch_ID'], how ='left')\
    .join(df_dealer,df_silver['Dealer_ID']==df_dealer['Dealer_ID'],how ='left')\
     .join(df_date,df_silver['Date_ID']==df_date['Date_ID'],how ='left')\
     .join(df_model,df_silver['Model_ID']==df_model['Model_ID'],how ='left')\
     .select(df_silver.Revenue, df_silver.Rev_Per_Unit,df_silver.Units_Sold,df_branch.dim_branch_key,df_dealer.dim_dealer_key, df_model.dim_model_key, df_date.dim_date_key)

In [0]:
df_fact.display()

In [0]:
#Incremental Run
 # Deduplicate source data based on merge keys
df_fact = df_fact.dropDuplicates(['dim_branch_key', 'dim_model_key', 'dim_date_key', 'dim_dealer_key'])
if spark.catalog.tableExists('cars_catalog.gold.fact_sales'):
    delta_table= DeltaTable.forPath(spark, 'abfss://gold@datalakeproject1goks.dfs.core.windows.net/fact_sales')
    delta_table.alias('trg').merge(df_fact.alias('src'), "src.dim_branch_key == trg.dim_branch_key and src.dim_model_key==trg.dim_model_key and src.dim_date_key==trg.dim_date_key and src.dim_dealer_key==trg.dim_dealer_key")\
                            .whenMatchedUpdateAll()\
                            .whenNotMatchedInsertAll()\
                            .execute()

#First Run
else:
    df_fact.write.format('delta')\
                  .mode('overwrite')\
                  .option("path",'abfss://gold@datalakeproject1goks.dfs.core.windows.net/fact_sales')\
                  .saveAsTable("cars_catalog.gold.fact_sales")

In [0]:
%sql
SELECT * FROM cars_catalog.gold.fact_sales